# インタラクティブに実行する

以下の流れで、PHYSBO をインタラクティブに実行することができます。

1. PHYSBO から次に実行するパラメータを得ます。
2. PHYSBO の外部で評価値を得ます。
3. 評価値をPHYSBOに登録します。

例えば、以下の様な場合に適しています。

- 人手による実験を行い、その評価値をPHYSBOに与えたい。
- simulator の実行を別プロセスで行うなど、柔軟に実行制御を行いたい。

## 探索候補データの準備

これまでのチュートリアルと同様、データセットファイル [s5-210.csv](https://raw.githubusercontent.com/issp-center-dev/PHYSBO/master/examples/grain_bound/data/s5-210.csv) を保存し、次のように読み出します。

In [1]:
import physbo

import numpy as np


def load_data():
    A =  np.asarray(np.loadtxt('s5-210.csv',skiprows=1, delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = physbo.misc.centering(X)

Cythonized version of physbo is used


## simulator の定義

In [2]:
class Simulator:
    def __init__( self ):
        _, self.t = load_data()
    
    def __call__( self, action ):
        return self.t[action]

## 最適化の実行

In [3]:
# policy のセット 
policy = physbo.search.discrete.Policy(test_X=X)

# シード値のセット 
policy.set_seed( 0 )

各探索ステップでは以下の処理を行っています。

1. `max_num_probes=1, simulator=None` として random_search または bayes_search を実行して action ID (パラメータ)を得る。
2. `t  = simulator(actions)` により評価値(の array) を得る。
3. `policy.write(actions, t)`により action ID (パラメータ) に対する評価値を登録する。
4. `physbo.search.utility.show_search_results` により履歴を表示する。

以下では、ランダムサンプリングを２回（1st, and 2nd steps）、ベイズ最適化による提案を２回（3rd, and 4th steps）を行います。

In [4]:
simulator = Simulator()

''' 1st step (random sampling) '''
actions = policy.random_search(max_num_probes=1, simulator=None)
t  = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

''' 2nd step (random sampling) '''
actions = policy.random_search(max_num_probes=1, simulator=None)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

''' 3rd step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, simulator=None, score='EI', interval=0,  num_rand_basis = 5000)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

''' 4-th step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, simulator=None, score='EI', interval=0,  num_rand_basis = 5000)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

interactive mode starts ... 
 
current best f(x) = -1.070602 (best action = 15673) 
list of simulation results
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=-1.070602 (action = 15673)


current best f(x) = -1.070602 (best action = 15673) 
list of simulation results
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=-1.070602 (action = 15673)
f(x)=-1.153410 (action = 16489)


Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood -3.530887145012235
50 -th epoch marginal likelihood -3.530886068830362
100 -th epoch marginal likelihood -3.530887353880807
1

## 中断と再開

以下の predictor, training, history を外部ファイルに保存することで、最適化プロセスを中断し、途中から再開することができます。

- predictor: 目的関数の予測モデル
- training: predictor の学習に用いるデータ (`physbo.Variable` オブジェクト）
- history: 最適化実行の履歴 (`physbo.search.discrete.History` オブジェクト)

In [5]:
policy.save(file_history='history.npz', file_training='training.npz', file_predictor='predictor.dump')

In [6]:
# policy を削除
del policy

# 保存した policy をロード 
policy = physbo.search.discrete.Policy(test_X=X)
policy.load(file_history='history.npz', file_training='training.npz', file_predictor='predictor.dump')

''' 5-th step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, simulator=None, score='EI', interval=0,  num_rand_basis = 5000)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

# predictor と training を個別に指定することも可
''' 6-th step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1,
                                            predictor=policy.predictor, training=policy.training,
                                            simulator=None, score='EI', interval=0,  num_rand_basis = 5000)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood -5.282317837949537
50 -th epoch marginal likelihood -5.314394752726803
100 -th epoch marginal likelihood -5.333676604278033
150 -th epoch marginal likelihood -5.344148457109117
200 -th epoch marginal likelihood -5.349388232794448
250 -th epoch marginal likelihood -5.351839835112543
300 -th epoch marginal likelihood -5.352933499846448
350 -th epoch marginal likelihood -5.353415283021385
400 -th epoch marginal likelihood -5.3536398281059405
450 -th epoch marginal likelihood -5.3537614274261145
500 -th epoch marginal likelihood -5.353841697978967
Done

current best f(x) = -0.985424 (best action = 8061) 
list of simulation results
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=0.000000 (action = 0)
f(x)=-1.070602 (action = 15673)
f(x)=-1.153410 (action = 16489)
f(x)=-0.993712 (action = 11874)
f(x)=-0.9